In [1]:
import sys

import numpy as np
import pysolr
from gensim.models import Doc2Vec

np.random.seed(42)
import smart_open
import pandas as pd
import gensim

In [2]:
# In case your sys.path does not contain the base repo, cd there.
print(sys.path)
%cd 'PATH_OF_BASE_REPO'  # In the solution it will be the path to my repo. This is such that python loads al the files from the top.

['/Users/axelsirota/repos/ml-solr-course/2-ranking/lab5', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python37.zip', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7/lib-dynload', '', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/IPython/extensions', '/Users/axelsirota/.ipython']
/Users/axelsirota/repos/ml-solr-course


In [3]:
model_path = '2-ranking/lab4/airbnb_model'
query = 'Midtown sunny chateau'
number_of_initial_retrieved = 100
model = None # Load the Doc2Vec model from lab4
print(f'Model loaded')

Model loaded


In [5]:
#Instantiate the client
solr = None

In [6]:
#Search for the query
results = []
print(f'Number of results were {len(results)}')

Number of results were 100


In [7]:
# Use the same simple_preprocess from the last lab 4 to tokenize the query
tokenized_query = list()

In [8]:
tokenized_query

['midtown', 'sunny', 'chateau']

In [9]:
inferred_vector = model.infer_vector(tokenized_query)
print(inferred_vector)

[ 0.04611075 -0.01602785 -0.05042642  0.00813507  0.08141993 -0.05650167
 -0.04308004  0.02841107  0.11073185  0.00939624 -0.08110334  0.03788716
  0.08754482 -0.00233287  0.04030268  0.00047163  0.0081155   0.07037532
 -0.08151705  0.10196422  0.01592677 -0.00575189 -0.12328304 -0.0444117
  0.05165997 -0.02518846  0.00267151 -0.06886391 -0.03039369  0.01192806
 -0.04475749  0.05799579 -0.02835805  0.00595889 -0.02112346  0.01466438
  0.02133428  0.02144625 -0.00342025  0.03254068 -0.05119841 -0.0741892
  0.07039563  0.01408158  0.0570154   0.00671781 -0.07529804 -0.00776306
  0.04450541 -0.08031283  0.00220784  0.0538672   0.02823217  0.10577902
  0.05967656 -0.00432712  0.0526036   0.01252659  0.03371542 -0.04590331
  0.06799655  0.05211353  0.0330506  -0.02344496  0.06321384  0.01104139
 -0.04707532  0.0724569   0.11450286  0.06106732 -0.01568583  0.02030839
 -0.04875635 -0.05453167 -0.01567344 -0.02905013  0.04325688 -0.05714364
  0.00621019 -0.01695368 -0.03705302  0.02718671  0.0

In [10]:
df_results = pd.DataFrame(results)
similarities = []
for result in results:
    similarity = 0 # Find the similarity between the query and the result using gensim similarity_unseen_docs method
    similarities.append(similarity)
df_results["Similarity"] = pd.Series(similarities)  # We store the similarities to order

Ok, what we have done is do the query against Solr, then finding the similarity between the descriptions of the query and the results.

The idea behind the algorithm would be to reorder the results based on the similarity score, not on BM25. Let's see which one is better.


In [11]:
df_results.head()

,id,listing_url,name,description,neighborhood_overview,host_id,host_url,host_name,neighbourhood,neighbourhood_cleansed,...,accommodates,bathrooms_text,bedrooms,beds,amenities,price,number_of_reviews,reviews_per_month,_version_,Similarity
0,10684164,https://www.airbnb.com/rooms/10684164,Private 1 Bdrm Apt - Prospect Park,"Located in the beautiful Chateau Frontenac, Br...","A nice retreat from the city, this area of Pro...",19227044,https://www.airbnb.com/users/show/19227044,Adam,"Brooklyn, New York, United States",Flatbush,...,4,1 bath,1.0,2.0,"[""Kitchen"", ""TV"", ""Essentials"", ""Washer"", ""Dry...",$75.00,1,[0.02],1706113109094039552,0.290233
1,4248788,https://www.airbnb.com/rooms/4248788,W70s Lg. Studio with outdoor space,Welcome to the chateau 70-something! This larg...,Charming townhouse on tree-lined Street. <br /...,22047286,https://www.airbnb.com/users/show/22047286,Chloe,"New York, United States",Upper West Side,...,2,1 bath,NaN,1.0,"[""Heating"", ""TV"", ""Fire extinguisher"", ""Carbon...",$250.00,5,[0.07],1706113107789611008,0.264746
2,48270841,https://www.airbnb.com/rooms/48270841,Romantic East Village Getaway,Welcome to the french chateau in the trees. Th...,The apartment is located in the heart of the E...,202489613,https://www.airbnb.com/users/show/202489613,Louis,"New York, United States",Gramercy,...,2,1 bath,1.0,1.0,"[""Kitchen"", ""TV"", ""Hangers"", ""Washer"", ""Essent...",$120.00,0,NaN,1706113115864694787,0.268404
3,39920510,https://www.airbnb.com/rooms/39920510,Large Elevator 1 Bedroom Best Location,Sunny & Spacious furnished apartment with an a...,NaN,61391963,https://www.airbnb.com/users/show/61391963,Stay With Vibe,NaN,Midtown,...,3,1 bath,1.0,2.0,"[""Essentials"", ""Dryer"", ""Long term stays allow...",$87.00,4,[0.26],1706113113706725380,0.467339
4,15721847,https://www.airbnb.com/rooms/15721847,Colorful Artist Loft w. your own Ocean Oasis,"The Broadway Chateau, as it's known, is a uniq...",We are in a classic south Williamsburg nabe fi...,909710,https://www.airbnb.com/users/show/909710,Claire,"Brooklyn, New York, United States",Williamsburg,...,1,1 shared bath,1.0,1.0,"[""Dishes and silverware"", ""Smoke alarm"", ""Stov...",$49.00,17,[0.32],1706113109836431361,0.195381


In [12]:
a = None # Sort the df_results by similarity column in descending order

In [13]:
a = a[:10].reset_index(drop=True)

In [15]:
print(f'Most similar document after reranking within retrieved results has description: \n\n{a["description"].iloc[0]}\nWith similarity: {a["Similarity"].iloc[0]}')

Most similar document after reranking within retrieved results has description: 

Sunny, contemporary, spacious and newly renovated 2-bedroom Astoria apartment, one block from the Astoria-Ditmars N/W subway stop. <br /><br />Just 15 minutes on the subway to midtown Manhattan! Hardwood floors throughout, large windows, full kitchen and living room, bathroom and beautiful bedroom, all within a gated community in a safe area.<br /><br />Spend your visit to NYC living like the locals with quick access to midtown Manhattan and several hidden treasures in the historic, trendy Astoria neighborhood.
With similarity: 0.5624656081199646


In [17]:
print(f'Most similar document before reranking within retrieved results has description: \n\n{df_results["description"].iloc[0]}\nWith similarity: {df_results["Similarity"].iloc[0]}')


Most similar document before reranking within retrieved results has description: 

Located in the beautiful Chateau Frontenac, Brooklyn. Built in the 1920s, this building offers a taste of Brooklyn's old school charm and easy access to Prospect Park. 750 Sq Ft, with large bedroom, living room and kitchen. Lots of sunlight.<br /><br /><b>The space</b><br />Located in the Flatbush neighborhood of Brooklyn and built circa 1930, The Chateau Frontenac Apartments bring the grandeur of a Manhattan-style courtyard residence to middle-class Brooklyn. Designed by Architect William Thomas McCarthy, these were some of the last single-family homes built in Park Slope; and some of the most iconic housing projects in New York City.<br /><br /><b>Guest access</b><br />You will have full access to the apartment's living spaces - large bedroom, kitchen, living room, and bathroom.
With similarity: 0.29023319482803345


In [16]:
print(f'Number of documents that surpass 0.5 similarity threshold: {len(a[a["Similarity"] >= 0.5])}')

Number of documents that surpass 0.5 similarity threshold: 8


It is remarkable how using DBOW the most similar result understood the need for midtown apartments that are chateaus. On the other hand the traditional top result ponderated chateau more just because it is a rare word.

It is not a perfect method, but a very good indication. A good idea is to have something like this **between** the raw results (thousands), filter them by similarity (hundreds) and then have a learning to rank recommender (dozens).

Tensorflow has opensources TF Recommenders which is great to plug in as an algorithm **after** these results. But this alone would work just fine.